In [3]:
# add autoreload
%load_ext autoreload
%autoreload 2
import pandas as pd
import pyreadstat
import os

In [4]:
os.chdir('T:/laupodteam/AIOS/Bram/data')

In [60]:
argus = pd.read_csv('./Argus/joined_argus/celldyn_raw.csv', sep=';')[['fcs_id']]
argus = argus.assign(dataset='ARGUS')
argus = argus.drop_duplicates()

ucorbio = pd.read_sas('./Argus/external_validation/ucorbio/st2_celldyn_13sep2021.sas7bdat')[['fcs_id']]
ucorbio = ucorbio.assign(dataset='UCORBIO')
ucorbio = ucorbio.assign(fcs_id=ucorbio.fcs_id.str.decode('utf-8'))
ucorbio = ucorbio.drop_duplicates()

heartscore = pd.read_csv('./HeartScore/Data/full_celldyn.csv', sep=';')[['fcs_id']]
heartscore = heartscore.assign(dataset='HEARTSCORE')
heartscore = heartscore.drop_duplicates()

MDS = pd.read_sas('./MDS/celldyn_sapph_20220905.sas7bdat')[['fcs_id']]
MDS = MDS.assign(dataset='MDS')
MDS = MDS.assign(fcs_id=MDS.fcs_id.str.decode('utf-8'))
MDS = MDS.drop_duplicates()

Reuma, _ = pyreadstat.read_sas7bdat('./Reuma/DT2RA/celldyn.sas7bdat')
Reuma = Reuma[['fcs_id']]
Reuma = Reuma.assign(dataset='REUMA')
Reuma = Reuma.drop_duplicates()

MDL, _ = pyreadstat.read_sas7bdat('./MDL/sapphire_19aug2021.sas7bdat')
MDL = MDL[['fcs_id']]
MDL = MDL.assign(dataset='MDL')
MDL = MDL.drop_duplicates()

LABELS = pd.concat([argus, ucorbio, heartscore, MDS, Reuma, MDL], axis=0)
LABELS = LABELS.assign(fcs_id=LABELS.fcs_id.str.strip())
LABELS = LABELS[LABELS.fcs_id.str.strip()!='']

C:\Users\bes3\AppData\Local\Temp/ipykernel_17000/3271673646.py:1: DtypeWarning: Columns (3,4,17,210) have mixed types. Specify dtype option on import or set low_memory=False.
  argus = pd.read_csv('./Argus/joined_argus/celldyn_raw.csv', sep=';')[['fcs_id']]


In [48]:
celldyn = pd.read_feather("./CellDyn/celldyn.feather")
celldyn = celldyn[['studyId_Alle_celldyn', 'fcs_id']]
celldyn = celldyn.assign(fcs_id=celldyn.fcs_id.str.decode('utf-8'))
celldyn = celldyn.set_index('fcs_id')

In [61]:
cnts = LABELS.groupby('fcs_id').size()
mults = cnts[cnts>1].index.tolist()

In [65]:
LABELS = LABELS.groupby('fcs_id').dataset.agg(lambda x: "_".join(set(x))).to_frame()

In [77]:
celldyn.join(LABELS, how='left').reset_index().to_parquet('./CellDyn/artifacts/celldyn_labels.parquet')